# JULIA MPI First Example: pi computaton

First step was to load MPI on my mac.  Seems mpich and openmpi are two reasonable choices
with probably no beginner's reason to prefer one over the other. <br>

I did  <t> brew install gcc </t> first to get the gcc compiler.  I ran into problems.  
The magic thing that told me what to do was <t> brew doctor </t>.  It wanted me to type
<t> xcode-select --install </t> and when I did, all was good.  I then typed
<t> brew install mpich </t> and mpi was just working.

My first example was to reproduce <a href="http://www.mcs.anl.gov/research/projects/mpi/tutorial/mpiexmpl/src/pi/C/solution.html">
the classic mypi </a> in the notebook

In [1]:
# Pkg.add("MPI")

In [1]:
using MPI

In [2]:
m = MPIManager(np=8)

MPI.MPIManager(np=8,launched=false,mode=MPI_ON_WORKERS)

In [3]:
addprocs(m)
#@mpi_do m comm = MPI.COMM_WORLD

8-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8
 9

In [4]:
@mpi_do m comm = MPI.COMM_WORLD
#
# Enter number of intervals, and tell every processor
# Traditional MPI would do this with a BCAST
#@mpi_do m n = 45

In [9]:
# Let's see if the processors got it
@mpi_do m println(n)

	From worker 2:	45
	From worker 3:	45
	From worker 5:	45
	From worker 9:	45
	From worker 7:	45
	From worker 4:	45
	From worker 6:	45
	From worker 8:	45


In [5]:
# my MPI id
@mpi_do m myid = MPI.Comm_rank(comm)
@mpi_do m println(myid)

	From worker 9:	7
	From worker 8:	6
	From worker 2:	0
	From worker 4:	2
	From worker 3:	1
	From worker 5:	3
	From worker 6:	4
	From worker 7:	5


In [6]:
# Get the number of processors
@mpi_do m np = MPI.Comm_size(comm)
@mpi_do m println(np)

	From worker 8:	8
	From worker 9:	8
	From worker 2:	8
	From worker 4:	8
	From worker 3:	8
	From worker 7:	8
	From worker 5:	8
	From worker 6:	8


Compute $\int_0^1 4/(1+x^2) dx= 4 atan(x)]_0^1$ which evaluates to $\pi$

In [7]:
using Interact

In [13]:
@time @mpi_do m begin
    n = 50_000_000
    comm = MPI.COMM_WORLD
    s = 0.0
    for i = MPI.Comm_rank(comm) + 1 : MPI.Comm_size(comm) : n 
        x = (i - .5)/n 
        s += 4/(1 + x^2) 
    end
    mypi = s/n
    our_π = MPI.Reduce(mypi, MPI.SUM, 0, comm)
    if myid==0
        println(our_π - π) 
    end
end

	From worker 2:	-8.881784197001252e-14
  3.913599 seconds (11.53 k allocations: 528.314 KB)


In [14]:
@fetchfrom 2 our_π

1.5707963567946053e8

In [15]:
@time begin
    n = 50_000_000
    h = 1/n
    our_π = 0
    for i = 0:h:1
        our_π += 4/(1 + i^2)
    end
    our_π*h
end

  9.786553 seconds (350.00 M allocations: 5.960 GB, 5.12% gc time)


3.1415927135892106

In [16]:
function f(n)

 @mpi_do m begin
    comm = MPI.COMM_WORLD
        
       
    s = 0.0
    for i = MPI.Comm_rank(comm) + 1 : MPI.Comm_size(comm) : $n 
        x = (i - .5)/$n 
        s += 4/(1 + x^2) 
    end
    mypi = s/$n
    our_π = MPI.Reduce(mypi, MPI.SUM, 0, comm)
    #if myid==0
     #   println(our_π - π) 
   # end
end
@fetchfrom 2 our_π   
end

f (generic function with 1 method)

In [17]:
f(100)

3.141600986923125

In [18]:
f(1000)

3.1415927369231262

In [19]:
f(10000)

3.1415926544231247

In [ ]:
@manipulate for n=10.^(3:10)
    f(n)
end

In [ ]:
using Interact

In [ ]:
@manipulate for i=1:10
    i
end

In [ ]:
1